In [14]:
# Dataset testing
import h5py
import numpy as np

# curr_dir = os.path.dirname(os.path.abspath(__file__))
#parent_dir = os.path.dirname(curr_dir)
#sys.path.append(parent_dir)

file_path = "../data/external/main_data.hdf5"

# ---------------- VARIABLES TO STORE
all_actions = []
all_observations = []
all_rewards_to_go = []
all_time_to_go = []
# -----------------------------------

with h5py.File(file_path, "r+") as f:
    for key in f.keys():
        #del episode_87["key-id"] removing a thing
        
        curr_episode = f[key]

        # split observations into the useful and useless observations (1 more observation than action) if not done before
        if "removed_last_obervation" not in curr_episode.keys():
            curr_episode["removed_last_obervation"] = curr_episode["observations"][-1]
            curr_episode["observations_data"] = curr_episode["observations"][:-1]

        # Inspect data in each attribute
        actions = curr_episode["actions"][:]
        infos = curr_episode["infos"]
        observations = curr_episode["observations_data"][:]
        rewards = curr_episode["rewards"][:]
        terminations = curr_episode["terminations"][:]
        truncations = curr_episode["truncations"][:]

        reward_sum = np.sum(rewards)
        rewards_to_go = np.zeros_like(rewards)
        time_to_go = np.zeros_like(rewards)
        for i in range (len(rewards)):
            rewards_to_go[i] = reward_sum
            reward_sum -= rewards[i]
            time_to_go[i] = len(rewards) - i
        # save the rewards as labels
        
        if "rewards_to_go" not in curr_episode.keys():
            curr_episode["rewards_to_go"] = rewards_to_go
            curr_episode["time_to_go"] = time_to_go

        # save the stuff
        all_actions.append(actions)
        all_observations.append(observations)
        all_rewards_to_go.append(rewards_to_go)
        all_time_to_go.append(time_to_go)
        

    print(rewards[-5:], rewards_to_go[-5:], time_to_go[-5:])
    # Dimensions:
    print(f"Keys in {key}:", list(curr_episode.keys()))
    print(f"{key} Action dimensions: {actions.shape}")
    print(f"Observation dimensions: {observations.shape}")
    print(f"Reward dimensions: {rewards.shape}")
    print(f"Termination dimensions: {terminations.shape}")
    print(f"Truncation dimensions: {truncations.shape}")

[4.53705551 4.75870536 5.20804495 5.30901434 4.93248319] [24.74530334 20.20824784 15.44954248 10.24149753  4.93248319] [5. 4. 3. 2. 1.]
Keys in episode_999: ['actions', 'infos', 'observations', 'observations_data', 'removed_last_obervation', 'rewards', 'rewards_to_go', 'terminations', 'time_to_go', 'truncations']
episode_999 Action dimensions: (1000, 8)
Observation dimensions: (1000, 105)
Reward dimensions: (1000,)
Termination dimensions: (1000,)
Truncation dimensions: (1000,)


In [15]:
print(len(all_actions))

1078


In [16]:
# combine the data into single vectors
all_actions = np.concatenate(all_actions, axis=0)
all_observations = np.concatenate(all_observations, axis=0)
all_rewards_to_go = np.concatenate(all_rewards_to_go, axis=0)
all_time_to_go = np.concatenate(all_time_to_go, axis=0)

# create a new dataset with all the values at once
output_file_path = "../data/processed/concatenated_data.hdf5"

with h5py.File(output_file_path, "w") as f:
    group = f.create_group("concatenated_data")

    # Save the concatenated data into the new group
    group.create_dataset("actions", data=all_actions)
    group.create_dataset("observations", data=all_observations)
    group.create_dataset("rewards_to_go", data=all_rewards_to_go)
    group.create_dataset("time_to_go", data=all_time_to_go)

print(f"Concatenated data saved under 'concatenated_data' in {output_file_path}")
print(f"Actions shape: {all_actions.shape}")
print(f"Observations shape: {all_observations.shape}")

Concatenated data saved under 'concatenated_data' in ../data/processed/concatenated_data.hdf5
Actions shape: (999382, 8)
Observations shape: (999382, 105)
